# **Convolution**

### Import des librairies

In [ ]:
%tensorflow_version 2.x
from sklearn.preprocessing import normalize
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

### Charger les données MNIST

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# réduire la taille du train pour accélérer les experiences
nnn=1000
x_train = x_train[:nnn]
y_train = y_train[:nnn]

print("x_train",x_train.shape)
print("Nombre d'image pour entrainer:",x_train.shape[0])
print("Nombre d'image pour tester:",x_test.shape[0])
print("La taille d'une image:",x_train.shape[1:],"ce qui fait au total:",
      x_train.shape[1]*x_train.shape[2],"pixels")
print("La liste des classes:",np.unique(y_train))

### Définition d'une fonction qui transforme en représentation binaire pour plusieurs classes  

In [ ]:
def transform_labels(y_train,y_test):
  """
  Cette fonction transform les classes non-binaire à une représentation binaire
  Par exemple si on a une liste de 6 fleures chacune peut avoir une des 3 classes
  Entrée: [
           1,
           3,
           3,
           2,
           1,
           2
          ]

  Sortie: [
           [1,0,0], # class 1
           [0,0,1], # class 3
           [0,0,1], # class 3
           [0,1,0], # class 2
           [1,0,0], # class 1
           [0,1,0]  # class 2
          ]
  """

  print('y_train',y_train.shape)
  print('y_test',y_test.shape)

  # concatener train et test
  y_train_test = np.concatenate((y_train,y_test),axis =0)

  # init un encoder Label
  encoder = LabelEncoder()
  # transformer de [1,3,3,2,1,2] à [0,2,2,1,0,1]
  new_y_train_test = encoder.fit_transform(y_train_test)

  # init un encoder one-hot
  encoder = OneHotEncoder()
  # transformer de [0,2,2,1,0,1] à la représentation binaire
  new_y_train_test = encoder.fit_transform(new_y_train_test.reshape(-1,1))

  # resplit the train and test
  new_y_train = new_y_train_test[0:len(y_train)]
  new_y_test = new_y_train_test[len(y_train):]

  print('new_y_train',new_y_train.shape)
  print('new_y_test',new_y_test.shape)

  return new_y_train.toarray(), new_y_test.toarray()

### Visualiser quelques images en spécifiant que c'est du noir et blanc

In [ ]:
plt.subplot(221)
plt.imshow(x_train[0], cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(x_train[1], cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(x_train[2], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(x_train[3], cmap=plt.get_cmap('gray'))
# show the plot
plt.show()

### Normaliser les données pour que chaque pixel soit entre 0 et 1

In [ ]:
# diviser par 255 pour que tout soit entre 0 et 1
x_train = x_train/255
x_test = x_test / 255

## Entrainer le réseau de neurones convolutif qui correspond au slide 36

### Transformer les classes en représentation binaire (one-hot encoding)

In [ ]:
y_train_binaire,y_test_binaire = transform_labels(y_train,y_test)

### Transformer la forme des images (ajouter une dimension vide pour dire qu'on ait un seul canal - gris au lieu de trois pour RGB comme les images couleurs)

In [ ]:
print(x_train.shape)
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)
print(x_train.shape)
# même chose pour le test set
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)

### Créer la couche d'entrée qui a la même shape que celle d'une instance dans `x_train`

In [ ]:
input_shape = x_train.shape[1:]

input_layer = keras.layers.Input(input_shape)

### Créer une couche convolutive cachée qui contient 8 filtres avec l'activation `relu`

In [ ]:
# padding: 'valid' -> on ne remplis pas l'image en entrée donc la convolution réduit la taille de l'image
#          'same'  -> on remplis l'image en entrée pour garder sa taille après la convolution
padding = 'same'

# spécifier la taille du stride
stride = 1

# spécifier la taille du filtre (ou kernel): (3,3) signifie 3x3
kernel_size = (3,3)

# spécifier le nombre de filtres
filters = 8

# spécifier l'activation ReLU
activation = 'relu'

# créer la couche convolutive 2D en lui spécifiant les hyper-paramètres et la lier à la couche d'entrée
hidden_conv_layer_1 = keras.layers.Conv2D(filters=filters,
                                          kernel_size=kernel_size,strides=stride,
                                          padding=padding,activation=activation)(input_layer)

### Lier un max pooling à la couche convolutive

In [ ]:
# spécifier la taille du pooling: (2,2) signifie 2x2
pool_size = (2,2)

# spécifier le stride
stride = 2

# spécifier le padding
padding = 'valid'

# créer l'opération Max Pooling pour 2D (pour les images c'est souvent 2D)
pooling_conv_layer_1 = keras.layers.MaxPooling2D(pool_size = pool_size, strides = stride,
                                          padding=padding)(hidden_conv_layer_1)

### Aplatir la sortie du pooling

In [ ]:
# on transforme la sortie du pooling (qui est une image) en un vecteur
flattened_layer_1 = keras.layers.Flatten()(pooling_conv_layer_1)

### Créer la couche de sortie

In [ ]:
# Cette couche correspond à la clasification
# donc elle contient C neurones avec C étant le nombre de classes
# elle utilise la fonction d'activation softmax
# cette couche pren en entrée la couche apaltie (flattened)

nb_classes = y_train_binaire.shape[1]

output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(flattened_layer_1)

### Créer maintenant le modèle

In [ ]:
model = keras.models.Model(inputs=input_layer, outputs=output_layer)

### Visualiser les informations du modèle

In [ ]:
model.summary()

### Choisir l'algorithme d'optimisation avec un learning rate de 0.1

In [ ]:
learning_rate = 0.01
optimizer_algo = keras.optimizers.SGD(lr=learning_rate)

### Choisir la fonction de coût qu'on veut optimiser: (Categorical Cross-Entropy)

In [ ]:
cost_function = keras.losses.categorical_crossentropy

### Compiler le modèle en lui indiquant qu'on veut mesurer aussi l'accuracy

In [ ]:
model.compile(loss=cost_function,optimizer=optimizer_algo, metrics=['accuracy'])

### Spécifier le fait qu'on veut sauvegarder le meilleur modèle sur le valdiation set

In [ ]:
model_checkpoint = keras.callbacks.ModelCheckpoint('best-model.h5', monitor='val_loss', save_best_only=True)

## Entrainement

### Choisir le batch size et le nombre d'époques

In [ ]:
mini_batch_size = 256
nb_epochs = 100

### Entrainer en lui spécifiant d'utiliser une partie du train pour la validation des hyper-paramèteres

In [ ]:
percentage_of_train_as_validation = 0.3
history = model.fit(x_train,y_train_binaire,batch_size=mini_batch_size,
                    epochs=nb_epochs,verbose=False,
                    validation_split=percentage_of_train_as_validation,
                    callbacks=[model_checkpoint])

### Tracer la variation du taux d'erreur sur le train et sur le validation set en fonction du nombre d'epoques

In [ ]:
history_dict = history.history
loss_train_epochs = history_dict['loss']
loss_val_epochs = history_dict['val_loss']

plt.figure()
plt.plot(loss_train_epochs,color='blue',label='train_loss')
plt.plot(loss_val_epochs,color='red',label='val_loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.savefig('epoch-loss.pdf')
plt.show()
plt.close()

### Choisir le modèle sauveguardé

In [ ]:
model = keras.models.load_model('best-model.h5')

## **Exercices**

### Evaluer sur le train et sur le test le modèle choisi

In [ ]:
# Votre code ici

 ### Corrigé :

In [ ]:
loss,acc = model.evaluate(x_train,y_train_binaire,verbose=False)

print("L'accuracy sur l'ensemble du train est:",acc)

loss,acc = model.evaluate(x_test,y_test_binaire,verbose=False)

print("L'accuracy sur l'ensemble du test est:",acc)

### Construire le modèle du slide 38 en ignorant dropout

In [ ]:
# Votre code ici

 ### Corrigé :

In [ ]:
# restart keras and tesnorflow session
keras.backend.clear_session()

# cherche le nombre de classe
nb_classes = y_train_binaire.shape[1]

# créer la couhce d'entrée qui a la meme shape qu'un image
input_shape = x_train.shape[1:]
input_layer = keras.layers.Input(input_shape)

#créer la première couche de convolution et le lier à la couche d'entrée
# n'oublier pas de spécifier les hyper-paramètres :
# nombre de filtres, padding, la taille du filtre, le stride, l'activation
hidden_conv_layer_1 = keras.layers.Conv2D(filters=10, padding='valid',
                                          kernel_size=(5,5), strides=1,
                                         activation='relu')(input_layer)
# créer l'operation max pooling qui prend en entrée la première convolutioom
hidden_pooling_layer_1 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=2,
                                                  padding='valid')(hidden_conv_layer_1)
# créer la deuxième convolution qui prend en entrée le pooling précedent
# n'oublier pas de spécifier les hyperparamèteres
hidden_conv_layer_2 = keras.layers.Conv2D(filters=20, padding='valid',
                                         kernel_size=(5,5),strides=1,
                                         activation='relu')(hidden_pooling_layer_1)
# créer le max pooling qui est liée à la convolution précédente
hidden_pooling_layer_2 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=2,
                                            padding='valid')(hidden_conv_layer_2)

# ignorer le dropout à cette séance
# il faut maintenant aplatir l'image en utilisant le layer Flatten
# qui est lié au pooling précédent
flatenned_layer_2 = keras.layers.Flatten()(hidden_pooling_layer_2)

# créer la couche de sortie qui contient un nombre de neurones égale
# au nombre de classes du dataset
output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(flatenned_layer_2)

# créer le modèle en spécifiant input et output
model = keras.models.Model(inputs=input_layer, outputs=output_layer)

# choisir le taux d'apprentissage
learning_rate = 0.1

# choisir l'algorithme d'optimsation en lui spécifiant le taux d'aprentissage
optimizer_algo = keras.optimizers.SGD(lr=learning_rate)

# choisir la fonction de coût: categorical cross entropy
cost_function = keras.losses.categorical_crossentropy

# compiler le modèle en lui spécifiant qu'on veut surveiller l'accuracy
model.compile(loss=cost_function,optimizer=optimizer_algo, metrics=['accuracy'])

# choisir le batch size
mini_batch_size = 256

# choisir le nombre d'époque
nb_epochs = 100

# spécifier le model checkpoint (pour sauveguarder le meilleur modèle à chaque époque )
model_checkpoint = keras.callbacks.ModelCheckpoint('best-model.h5', monitor='val_loss', save_best_only=True)

# spécifier le pourcentage pour la validation
percentage_of_train_as_validation = 0.3

# commencer l'entrainement
history = model.fit(x_train,y_train_binaire,batch_size=mini_batch_size,epochs=nb_epochs,
                    validation_split=percentage_of_train_as_validation,verbose=False,
                   callbacks=[model_checkpoint])


### Tracer la variation de l'accuracy (non pas le coût) en fonction du nombre d'epoques (donc utiliser 'acc' au lieu de 'loss')

In [ ]:
# Votre code ici

 ### Corrigé :

In [ ]:
history_dict = history.history
loss_train_epochs = history_dict['accuracy']
loss_val_epochs = history_dict['val_accuracy']

plt.figure()
plt.plot(loss_train_epochs,color='blue',label='train_acc')
plt.plot(loss_val_epochs,color='red',label='val_acc')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()
plt.show()
plt.close()

### Afficher le summary de votre modèle et analyser la sortie (cette étape peut-être faite avant l'entrainement)

In [ ]:
# Votre code ici

 ### Corrigé :

In [ ]:
model.summary()

### Choisir le modèle sauveguardé

In [ ]:
# Votre code ici

 ### Corrigé :

In [ ]:
model = keras.models.load_model('best-model.h5')

### Évaluer le model

In [ ]:
# Votre code ici

 ### Corrigé :

In [ ]:
# evaluation sur train
loss,acc = model.evaluate(x_train,y_train_binaire,verbose=False)
print("L'accuracy sur l'ensemble du train est:",acc)

# evaluation sur le test
loss,acc = model.evaluate(x_test,y_test_binaire,verbose=False)
print("L'accuracy sur l'ensemble du test est:",acc)